In [1]:
# pip install -q transformers[torch] datasets
# pip install -q bitsandbytes trl peft
# pip install flash-attn --no-build-isolation

In [1]:
from datasets import DatasetDict
import json

In [2]:
filename = "llama-3-finetuning-data.jsonl"

In [3]:
data = []
with open(filename, 'r', encoding='utf-8') as f:
  for line in f:
    data.append(json.loads(line.strip()))


In [4]:
from datasets import Dataset

dataset = Dataset.from_list(data)

train_test_data = dataset.train_test_split(test_size=63, seed=42)
train_val_data = train_test_data["train"].train_test_split(test_size=63, seed=42)

# Access the train, validation, and test datasets
train_dataset = train_val_data["train"]
eval_dataset = train_val_data["test"]
test_dataset = train_test_data["test"]

# Print the number of examples in each set
print(f"Train dataset: {len(train_dataset)} examples")
print(f"Validation dataset: {len(eval_dataset)} examples")
print(f"Test dataset: {len(test_dataset)} examples")



Train dataset: 577 examples
Validation dataset: 63 examples
Test dataset: 63 examples


In [6]:
for item in test_dataset:
    print(item)

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.<|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\nDefinition of a Functional Area:\n- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.\n\nDefinition of Management Level:\n- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.\n- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."\n\nDefinition of a Job Ti

In [5]:
dataset_dict = {"train": train_dataset,
                "test": eval_dataset}

In [6]:
raw_datasets = DatasetDict(dataset_dict)

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 577
    })
    test: Dataset({
        features: ['text'],
        num_rows: 63
    })
})

In [8]:
from transformers import AutoTokenizer

model_name = "/home/raza/Downloads/meta-llama-3-2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048


In [9]:
from transformers import BitsAndBytesConfig
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, set_seed
from transformers import AutoProcessor, DataCollatorForSeq2Seq, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoConfig, GenerationConfig

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

processor = AutoTokenizer.from_pretrained(model_name)
processor.padding_side = 'right'

# Set the padding token to be the same as eos_token if it's not defined
if processor.pad_token is None:
    processor.pad_token = processor.eos_token  # or you can set a custom token if needed




model = AutoModelForCausalLM.from_pretrained(
          model_name,
          trust_remote_code=True,
          attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
          torch_dtype="auto",
          use_cache=False, # set to False as we're going to use gradient checkpointing
          device_map=device_map,
          quantization_config=quantization_config,
)





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def inference_function(input_text: str, max_length: int = 50):
    """
    Simple inference function to generate text based on an input prompt.

    Args:
    - input_text (str): The input text to be processed by the model.
    - max_length (int): The maximum length of the generated text.

    Returns:
    - str: The generated text.
    """
    # Process the input text
    inputs = processor(input_text, return_tensors="pt", padding=True).to(model.device)

    # Generate output from the model
    output = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,  # Enable sampling for more diverse outputs
        temperature=1.0,
        top_p=0.9
    )

    # Decode the output tokens into text
    generated_text = processor.decode(output[0], skip_special_tokens=True)  # Get the first sequence

    return generated_text

# Example usage
input_text = "Once upon a time"
generated_output = inference_function(input_text)
print(f"Generated Output: {generated_output}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Output: Once upon a time, in a bustling city, there was a beautiful and vibrant public park. The park was a hub of activity, with people from all walks of life coming to walk, jog, play games, or simply enjoy the scenery.


In [11]:
model = prepare_model_for_kbit_training(model)

In [12]:
total_epochs = 15

In [13]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = './finetuned_llama3-2-3B_model'


# Adjusted TrainingArguments for 15 epochs and saving checkpoints every 2 epochs
training_args = TrainingArguments(
    fp16=True,  # Use mixed precision
    bf16=False,  # BF16 is generally for specific hardware; here, fp16 is better suited
    do_eval=True,
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    evaluation_strategy="epoch",  # Ensure evaluation happens at every epoch
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate a larger batch size
    gradient_checkpointing=True,  # Save memory during training
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Optional, helps with stability
    learning_rate=2e-5,  # Low learning rate for fine-tuning
    log_level="info",
    logging_steps=10,  # Log every 10 steps
    logging_strategy="steps",
    lr_scheduler_type="cosine",  # Use a cosine decay scheduler
    max_steps=-1,  # Train for the full number of epochs
    num_train_epochs=total_epochs,  # Fine-tune for 15 epochs
    output_dir=output_dir,  # Path where the model is saved
    overwrite_output_dir=True,  # Overwrite any existing model
    per_device_train_batch_size=4,  # Batch size of 4 (adjust if necessary)
    per_device_eval_batch_size=4,  # Eval batch size (use 2-4 depending on memory)
    push_to_hub=False,
    save_strategy="epoch",  # Save model after each epoch
    save_steps=None,  # Disable saving based on steps (use epochs instead)
    save_total_limit=None,  # Limit the number of saved checkpoints (optional)
    seed=42,
    logging_dir="./logs",  # Directory to store logs (TensorBoard)
    load_best_model_at_end=True,  # Load the best model based on validation loss
)

# based on config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

trainer = SFTTrainer(
    model=model_name,
    args=training_args,
    train_dataset=raw_datasets["train"],
    eval_dataset=raw_datasets["test"],
    tokenizer=processor,
    peft_config=peft_config,
)




/home/raza/miniconda3/envs/finetuning_v2/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_165834/1722739665.py:49: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/home/raza/miniconda3/envs/finetuning_v2/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Using auto half precision backend


In [13]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# chat = [
#   {"role": "system", "content": "You are a sentiment analysis agent"},
#   {"role": "user", "content": "Classify the following statement as Positive or Negative:\nStatement: I am happy"},
#   {"role": "assistant", "content": "Positive"},
# ]

# tokenizer.apply_chat_template(chat, tokenize=False)

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
trainer_result = trainer.train()

***** Running training *****
  Num examples = 577
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 540
  Number of trainable parameters = 36,700,160


  0%|          | 0/540 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


{'loss': 2.4912, 'grad_norm': 0.28775086998939514, 'learning_rate': 1.9983081582712684e-05, 'epoch': 0.28}
{'loss': 2.3672, 'grad_norm': 0.31076881289482117, 'learning_rate': 1.9932383577419432e-05, 'epoch': 0.55}
{'loss': 2.2098, 'grad_norm': 0.3079773187637329, 'learning_rate': 1.9848077530122083e-05, 'epoch': 0.83}



***** Running Evaluation *****
  Num examples = 63
  Batch size = 4


...

  0%|          | 0/16 [00:00<?, ?it/s]

Saving model checkpoint to ./finetuned_llama3-2-3B_model/checkpoint-540
loading configuration file /home/raza/Downloads/meta-llama-3-2-3B-Instruct/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version":

{'eval_loss': 0.09833656251430511, 'eval_runtime': 4.6998, 'eval_samples_per_second': 13.405, 'eval_steps_per_second': 3.404, 'epoch': 14.61}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./finetuned_llama3-2-3B_model/checkpoint-540 (score: 0.09833656251430511).


{'train_runtime': 2147.4783, 'train_samples_per_second': 4.03, 'train_steps_per_second': 0.251, 'train_loss': 0.43018975059191383, 'epoch': 14.61}


In [15]:
train_result = trainer.train()

***** Running training *****
  Num examples = 577
  Num Epochs = 50
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 7,200
  Number of trainable parameters = 12,582,912


  0%|          | 0/7200 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 1.4911, 'grad_norm': 0.04986097291111946, 'learning_rate': 1.999997620177352e-05, 'epoch': 0.03}
{'loss': 1.4963, 'grad_norm': 0.05934108793735504, 'learning_rate': 1.9999904807207348e-05, 'epoch': 0.07}
{'loss': 1.4869, 'grad_norm': 0.0683630183339119, 'learning_rate': 1.9999785816641293e-05, 'epoch': 0.1}
{'loss': 1.4842, 'grad_norm': 0.07281941920518875, 'learning_rate': 1.9999619230641714e-05, 'epoch': 0.14}
{'loss': 1.465, 'grad_norm': 0.06812652945518494, 'learning_rate': 1.99994050500015e-05, 'epoch': 0.17}
{'loss': 1.4492, 'grad_norm': 0.06429293751716614, 'learning_rate': 1.999914327574007e-05, 'epoch': 0.21}
{'loss': 1.4394, 'grad_norm': 0.06659349054098129, 'learning_rate': 1.9998833909103385e-05, 'epoch': 0.24}
{'loss': 1.4123, 'grad_norm': 0.07142454385757446, 'learning_rate': 1.9998476951563914e-05, 'epoch': 0.28}
{'loss': 1.3984, 'grad_norm': 0.07648936659097672, 'learning_rate': 1.9998072404820648e-05, 'epoch': 0.31}
{'loss': 1.3829, 'grad_norm': 0.081494316458


***** Running Evaluation *****
  Num examples = 63
  Batch size = 2


...

Saving model checkpoint to ./finetuned_model/checkpoint-7200
loading configuration file /home/raza/Downloads/Phi-3-mini-4k-instruct/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "sliding_window": 2047,
  "tie_word_embeddings": false,
  "torch_dt

{'loss': 0.0219, 'grad_norm': 0.0829143151640892, 'learning_rate': 0.0, 'epoch': 49.66}


  0%|          | 0/32 [00:00<?, ?it/s]

Saving model checkpoint to ./finetuned_model/checkpoint-7200
loading configuration file /home/raza/Downloads/Phi-3-mini-4k-instruct/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "sliding_window": 2047,
  "tie_word_embeddings": false,
  "torch_dt

{'eval_loss': 0.04038238525390625, 'eval_runtime': 6.875, 'eval_samples_per_second': 9.164, 'eval_steps_per_second': 4.655, 'epoch': 49.66}
{'train_runtime': 9774.2836, 'train_samples_per_second': 2.952, 'train_steps_per_second': 0.737, 'train_loss': 0.05435840782605939, 'epoch': 49.66}
